In [ ]:
!pip install --upgrade keras tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/kaggle

/content/gdrive/MyDrive/kaggle


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
import pandas as pd


In [ ]:

# Load the CSV file into a DataFrame
data = pd.read_csv("/content/gdrive/MyDrive/kaggle/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", header=None, names=["target", "ids", "date", "flag", "user", "text"], delimiter=',')

# Display the first few rows of the DataFrame
print(data.head())
data = data.iloc[:len(data)//4]


   target         ids                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  


In [ ]:
# Drop columns except the 'text' column
text_data = data[['text']]

# Extract the 'target' column
labels = data['target']

In [ ]:
# Preprocess the text data
text_data = data['text'].astype(str)  # Ensure all values are strings
text_data = text_data.apply(lambda x: x.lower())  # Convert text to lowercase
text_data = text_data.str.replace(r'http\S+|www.\S+', '', regex=True)  # Remove URLs
text_data = text_data.str.replace(r'[^a-zA-Z\s]', '', regex=True)  # Remove punctuation and non-alphabetic characters

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)
max_sequence_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_sequence_length)


In [ ]:
print("Dimensions of X:", X.shape)

Dimensions of X: (400000, 35)


In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load data in chunks
#chunk_size = 100000
#chunks = pd.read_csv('/content/gdrive/MyDrive/kaggle/training.1600000.processed.noemoticon.cs', chunksize=chunk_size, encoding='latin-1', header=None,
                     #names=['target', 'id', 'date', 'flag', 'user', 'text'])

# Combine chunks into a single DataFrame
data = pd.read_csv("/content/gdrive/MyDrive/kaggle/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", header=None, names=["target", "ids", "date", "flag", "user", "text"], delimiter=',')

# Preprocess data
X = data['text']
y = data['target'].replace({4: 1})  # Convert target labels to binary (0 or 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define maximum sequence length and maximum number of words
max_sequence_length = 100
max_num_words = 10000



# Tokenize text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

# Pad sequences
max_sequence_length = 100
X_pad = pad_sequences(sequences, maxlen=max_sequence_length)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the CNN model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Binary classification, so output dimension is 2

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 256
model.fit(X_train, y_train, epochs=3, batch_size=batch_size, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Test Accuracy:", accuracy)
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 595s 119ms/step - accuracy: 0.7858 - loss: 0.4478 - val_accuracy: 0.8202 - val_loss: 0.3938
Epoch 2/3
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 558s 112ms/step - accuracy: 0.8338 - loss: 0.3697 - val_accuracy: 0.8250 - val_loss: 0.3864
Epoch 3/3
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 570s 114ms/step - accuracy: 0.8505 - loss: 0.3381 - val_accuracy: 0.8255 - val_loss: 0.3926
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 37s 30ms/step - accuracy: 0.8246 - loss: 0.3928
Test Accuracy: 0.8255468606948853


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (256, 100, 100)             │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (256, 98, 64)               │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (256, 49, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (256, 3136)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (256, 64)                   │         200,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (256, 2)                    │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,660,488 (13.96 MB)

 Trainable params: 1,220,162 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,440,326 (9.31 MB)

None
